# Notebook to find scraped charity names in the Panama Papers using Spark

In [1]:
# Imports
import re
import nltk
import json
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#stop words
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

#spark
import findspark
findspark.init('/opt/spark/spark-2.3.2-bin-hadoop2.7/')

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min
from pyspark.sql.functions import udf
from pyspark.sql.functions import split
from pyspark.sql.functions import explode

from pyspark.sql.types import StringType
from pyspark.sql.types import TimestampType

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()


In [2]:
# File paths
DATA_FOLDER = '../data'
PANAMA_DATA_FOLDER = DATA_FOLDER + '/panama'

GENERATED_FOLDER = '../generated'
CHARITY_GENERATED_FOLDER = GENERATED_FOLDER + '/charities'

In [3]:
# Loading and creation of dataframes
pp_edges = spark.read.csv(PANAMA_DATA_FOLDER + '/panama_papers.edges.csv', header=True)
pp_adress = spark.read.csv(PANAMA_DATA_FOLDER + '/panama_papers.nodes.address.csv', header=True)
pp_entity = spark.read.csv(PANAMA_DATA_FOLDER + '/panama_papers.nodes.entity.csv', header=True)
pp_intermediary = spark.read.csv(PANAMA_DATA_FOLDER + '/panama_papers.nodes.intermediary.csv', header=True)
pp_officer = spark.read.csv(PANAMA_DATA_FOLDER + '/panama_papers.nodes.officer.csv', header=True)

wiki_info = spark.read.csv(CHARITY_GENERATED_FOLDER + '/wikipedia_charity_info.csv', header=True)
wiki_links =charities_info = spark.read.csv(CHARITY_GENERATED_FOLDER + '/wikipedia_charity_links.csv', header=True)

In [4]:
# Addition of english stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

stop_words.add('&')
stop_words.add('co')
stop_words.add('co.')
stop_words.add('co.,')
stop_words.add('co.,ltd.')
stop_words.add('corp')
stop_words.add('corp.')
stop_words.add('corp.,')
stop_words.add('de')
stop_words.add('inc.')
stop_words.add('foundation')
stop_words.add('inc')
stop_words.add('limited')
stop_words.add('ltd')
stop_words.add('ltd.')
stop_words.add('s.a.')

stop_words = list(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sabrina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Splitting each charity name and cleaning stop words

def to_lower_parens_less(word):
    return word.lower().replace('(', '').replace(')', '')

charity_name = wiki_info.select('Name').selectExpr('Name as CharityName')
def remove_stop(word_list):
    return [to_lower_parens_less(w) for w in word_list if w.lower() not in stop_words]
    
# Extracting company shell names

shell_name=pp_entity.select('name').selectExpr('name as ShellName')

In [6]:
charity_name.head(5)

[Row(CharityName='AARP Foundation'),
 Row(CharityName="Acorns Children's Hospice"),
 Row(CharityName='Action Against Hunger'),
 Row(CharityName='Action Deafness'),
 Row(CharityName='ActionAid')]

In [7]:
shell_name.head(5)

[Row(ShellName='TIANSHENG INDUSTRY AND TRADING CO., LTD.'),
 Row(ShellName='NINGBO SUNRISE ENTERPRISES UNITED CO., LTD.'),
 Row(ShellName='HOTFOCUS CO., LTD.'),
 Row(ShellName='SKY-BLUE GIFTS & TOYS CO., LTD.'),
 Row(ShellName='FORTUNEMAKER INVESTMENTS CORPORATION')]

In [8]:
type(shell_name)

pyspark.sql.dataframe.DataFrame

In [9]:
charities_vs_shells = shell_name.crossJoin(charity_name)

In [10]:
charities_vs_shells.head(5)

[Row(ShellName='TIANSHENG INDUSTRY AND TRADING CO., LTD.', CharityName='AARP Foundation'),
 Row(ShellName='TIANSHENG INDUSTRY AND TRADING CO., LTD.', CharityName="Acorns Children's Hospice"),
 Row(ShellName='TIANSHENG INDUSTRY AND TRADING CO., LTD.', CharityName='Action Against Hunger'),
 Row(ShellName='TIANSHENG INDUSTRY AND TRADING CO., LTD.', CharityName='Action Deafness'),
 Row(ShellName='TIANSHENG INDUSTRY AND TRADING CO., LTD.', CharityName='ActionAid')]

In [19]:
def check_for_words(charity, shell, percentage, lower_bound):
    if charity is None or shell is None:
        return False
    
    charity_words = [x.lower() for x in charity.split()]
    shell_words = [x.lower() for x in shell.split()]
    len_charity = len(charity_words)
    len_shell = len(shell_words)
    
    count = 0
    for word in charity_words:
        if word in shell_words and word.lower() not in stop_words:
            count += 1
    return ((count/len_charity >= percentage) and (count/len_shell >= percentage))
        

In [20]:
filtered = charities_vs_shells.rdd.filter(lambda r: check_for_words(r[0], r[1], 0.5, 3) == True)
filtered.take(5)

[Row(ShellName='First Limited', CharityName=' first Director'),
 Row(ShellName='CAPITAL WORLD INTERNATIONAL LTD.', CharityName='World Vision International'),
 Row(ShellName='HARTFORD SHIPPING S. A.', CharityName='John A. Hartford Foundation'),
 Row(ShellName='COMMUNICATION SERVICES INTERNATIONAL INC.', CharityName="Holt International Children's Services"),
 Row(ShellName='WORLD WIN INTERNATIONAL LIMITED', CharityName='World Vision International')]

In [15]:
matches = filtered.toDF()

In [18]:
matches.show()

+--------------------+--------------------+
|           ShellName|         CharityName|
+--------------------+--------------------+
|Medical World Lim...|World Medical Relief|
|          FINCA INC.| FINCA International|
|Sona Internationa...|Holt Internationa...|
|Going This Way Li...|Born This Way Fou...|
|Magic Internation...|Holt Internationa...|
|Top International...|Holt Internationa...|
|Vega Internationa...|Holt Internationa...|
|Expert Internatio...|Holt Internationa...|
|World Internation...|World Vision Inte...|
|    Caneo Foundation|     AARP Foundation|
|    Caneo Foundation|  Adelson Foundation|
|    Caneo Foundation|Allegheny Foundation|
|    Caneo Foundation|     Bush Foundation|
|    Caneo Foundation| Carthage Foundation|
|    Caneo Foundation|  Clinton Foundation|
|    Caneo Foundation| Confetti Foundation|
|    Caneo Foundation|   Drosos Foundation|
|    Caneo Foundation|     Edhi Foundation|
|    Caneo Foundation| Epilepsy Foundation|
|    Caneo Foundation|   Eppley 